In [1]:
from ai71 import AI71
from dotenv import load_dotenv
import os

from extract_pdf import extract_text_from_pdf, extract_text_from_pdfs

In [2]:
load_dotenv()
api_key = os.getenv('API_KEY')

In [3]:
client = AI71(api_key)

In [4]:
text = extract_text_from_pdf("/home/space/Downloads/test.pdf")

In [7]:
multiple_texts = extract_text_from_pdfs(["/home/space/Downloads/test.pdf", "/home/space/Downloads/test.pdf"])

There is a limit of around 5500 characters for the input data. Maybe can use streaming to input all the data

In [17]:
response = client.chat.completions.create(
    model="tiiuae/falcon-180B-chat",
    messages=[
        {"role": "system", "content": "You are a lesson planner."},
        {"role": "user", "content": text[0:5500]},
        {"role": "user", "content": "Create lesson topics to teach the given material."},
    ],
    # max_tokens=100,
)

Chat completion and completion are virtually the same

In [ ]:
response = client.completions.create(
    model="tiiuae/falcon-180B-chat",
    prompt=f"{text[0:5500]}, Create lesson topics to teach the given material.",
)

In [31]:
print(response.choices[0].message.content)

 1. Introduction to Calculus
2. Vectorization and Linear Algebra
3. Statistics for Machine Learning
4. Descriptive Statistics: Mean, Median, Mode, Variance, and Standard Deviation
5. Gaussian Distribution and Central Limit Theorem
6. Confidence Intervals and Hypothesis Testing
7. Gradient Estimation and Gradient Descent



In [21]:
lessons = response.choices[0].message.content
lesson_list = response.choices[0].message.content.split("\n")
print(lessons)

 1. Introduction to Calculus
2. Vectorization and Linear Algebra
3. Statistics for Machine Learning
4. Descriptive Statistics: Mean, Median, Mode, Variance, and Standard Deviation
5. Gaussian Distribution and Central Limit Theorem
6. Confidence Intervals and Hypothesis Testing
7. Gradient Estimation and Gradient Descent



In [39]:
content_response = client.chat.completions.create(
    model="tiiuae/falcon-180B-chat",
    messages=[
        {"role": "system", "content": "You are a lesson creator."},
        {"role": "user", "content": text[0:5000]},
        {"role": "user", "content": lesson_list[0]},
        {"role": "user", "content": "Create lesson on the topics to teach the given material in a lot of detial."},
    ],
)

In [36]:
print(content_response)

id='RequestType.CHAT_COMPLETION-tiiuae/falcon-11b-99bf3161-4b4f-401f-8ac6-99a9712cd5b4-api71-api-99bf3161-4b4f-401f-8ac6-99a9712cd5b4-62501443-6db5-4157-a951-f44672165bae' created=1722128540 model='tiiuae/falcon-11b' usage=CompletionUsage(completion_tokens=3000, prompt_tokens=1828, total_tokens=4828) choices=[Choice(finish_reason=<FinishReason.STOP: 'stop'>, index=0, message=ChatCompletionMessage(content='\n\n\n Topic : Introduction to Statistics\nDescription : \n Introduction theory computing language\nPurpose : \nProvide defination \nGradient descent \n\nPrerequisite :\nCalculus Basics\n\n\nOutline : \n\nI. Introduction\n1. Definition of statistics\n2. Types of statistics\n3. Purpose of Statistics\n4. Need for statistics\n\nII. Population \n1. Definition of population\n2. Different types of populations and population set\n\ta. Discrete & continuous populations\n\nIII. Sample \n1. Definition of sample\n2. Different types of samples\n\ta. Probability \n\tb. Non probability\n\nIV. Stati

In [37]:
print(content_response.choices[0].message.content)




 Topic : Introduction to Statistics
Description : 
 Introduction theory computing language
Purpose : 
Provide defination 
Gradient descent 

Prerequisite :
Calculus Basics


Outline : 

I. Introduction
1. Definition of statistics
2. Types of statistics
3. Purpose of Statistics
4. Need for statistics

II. Population 
1. Definition of population
2. Different types of populations and population set
	a. Discrete & continuous populations

III. Sample 
1. Definition of sample
2. Different types of samples
	a. Probability 
	b. Non probability

IV. Statistic 
1. Definition of statistic
2. Types of statistics
	a. Measures of central tendency
	b. Measures of dispersion
	c. Position of the ordinal variable
	d. Measures of skewness

V. Analysis of Variance 
1. Concept of Analysis of Variance 
2. One way and Two way Analysis of Variance

VI. Correlation 
1. Concept of Correlation
2. Types of correlation
3. Introduction to T-test 


Defination : 
Definition of eects, analysis of variance signific